In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Deep Learning Challenge

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from data_preparation import load_data, delete_outliers
import numpy as np
from visualization import plot_history
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras.metrics import Precision, Recall
import matplotlib.pyplot as plt
import random
import cv2
import os

In [ ]:
# the ensemble of models
model_1 = tfk.models.load_model('Efficientnet_final')
model_2 = tfk.models.load_model('Xception_final')
model_3 = tfk.models.load_model('mobilenet_final')
model_4 = tfk.models.load_model('ConvNextModel')
model_5 = tfk.models.load_model('convnext_final')

In [ ]:
# renaming (different names are required)
model_2._name = model_2.name + str('_2')
model_3._name = model_3.name + str('_3')
model_4._name = model_4.name + str('_4')
model_5._name = model_5.name + str('_5')

In [ ]:
input_layer = tfkl.Input(shape = (96,96,3), name = 'input_layer')    # single input layer connected to each model

# models 1,4,5 are trained on normalized data: prepare data to be normalized when imported

branch_1 = model_1(input_layer)    # efficientnet has incorporated preprocessing

branch_2_preprocess = tf.keras.layers.Lambda(lambda x: x*255)(input_layer)    # assuming to receive normalized data: but preprocessing function need non normalized data
branch_2_preprocess = tf.keras.layers.Lambda(tfk.applications.xception.preprocess_input)(branch_2_preprocess)
branch_2 = model_2(branch_2_preprocess)

branch_3_preprocess = tf.keras.layers.Lambda(lambda x: x*255)(input_layer)    # assuming to receive normalized data: but preprocessing function need non normalized data
branch_3_preprocess = tf.keras.layers.Lambda(tfk.applications.mobilenet_v2.preprocess_input)(branch_3_preprocess)
branch_3 = model_3(branch_3_preprocess)

branch_4_preprocess = tf.keras.layers.Lambda(lambda x: x*255)(input_layer)    # assuming to receive normalized data: but preprocessing function need non normalized data
branch_4_preprocess = tf.keras.layers.Lambda(tfk.applications.convnext.preprocess_input)(branch_4_preprocess)
branch_4 = model_5(input_layer)

branch_5_preprocess = tf.keras.layers.Lambda(lambda x: x*255)(input_layer)    # assuming to receive normalized data: but preprocessing function need non normalized data
branch_5_preprocess = tf.keras.layers.Lambda(tfk.applications.convnext.preprocess_input)(branch_5_preprocess)
branch_5 = model_5(input_layer)

# some preptrained networks, such as mobilenet, xception, ... require preprocessing, that can directly be included separately for
# each model with a lambda layer, that contains a custom function

concat = tfkl.Add()([branch_1, branch_2, branch_3, branch_4, branch_5])    # adding the 5 outcomes
divide = tfkl.Lambda(lambda x: x/5)(concat)    # lambda layer to normalize the probabilities
final_model = tfk.Model(inputs=input_layer, outputs = divide)

In [ ]:
final_model.compile(tfk.optimizers.AdamW(), tfk.losses.CategoricalCrossentropy(), metrics = 'accuracy')
# formally needed to evaluate the accuracy or other functions while trying the model

In [ ]:
final_model.save('ensemble_final')